### note


slice out the RS ID by sample matrix for jisoo

In [2]:
### parameters
import pandas as pd
import numpy as np
mySpecie='Homo_sapiens'
"""
Download from synapse if it is not installed
"""
skymap_snp_dir='/cellar/users/btsui/Data/SRA/snp/{specie}_snp_pos/'.format(specie=mySpecie)

In [3]:
queryChr,queryPosition='7',140753336 

In [4]:
window=15
readThreshold=5

In [8]:
#!ls /cellar/users/btsui/Data/SRA/snp/Homo_sapiens_snp_pos/*

In [9]:
#skymap_snp_dir+'Pos_block_'+str(myChunk)

In [10]:
# b'Chr'

In [11]:
#myPosDf

In [12]:
%%time
chunkSize=100000 #fixed params
myChunk=int(queryPosition/chunkSize)*chunkSize
hdf_s=pd.HDFStore(skymap_snp_dir+'Pos_block_'+str(myChunk),mode='r')
tmpChunkDf=hdf_s['/chunk']
myPosDf=tmpChunkDf[(tmpChunkDf.Pos>=(queryPosition-window))&(tmpChunkDf.Pos<=(queryPosition+window))].reset_index()
myPosDf.columns=myPosDf.columns.astype(np.str)


CPU times: user 340 ms, sys: 340 ms, total: 680 ms
Wall time: 3.21 s


In [13]:
#tmpChunkDf

In [14]:
m1=myPosDf['Chr']==queryChr
m2=myPosDf['ReadDepth']>=readThreshold
myChrPosDf=myPosDf[m1&m2]

In [17]:
#myChunk=int(queryPosition/chunkSize)*chunkSize
#myChunk


In [18]:
print ('# of studies sliced out:' ,(tmpChunkDf.Run_digits.nunique()))

# of studies sliced out: 149064


###### find reference base for each chrosome coordinate

In [19]:
###it's a 
"""
The original vcf file can be downloaded from: ftp://ftp.ncbi.nih.gov/snp/organisms/human_9606/VCF/All_20170710.vcf.gz
I ignored all the entries without any phentoype related resources to suppport, where the byte2 of the VCF equals to zero.
For more info: ftp://ftp.ncbi.nlm.nih.gov/snp/specs/dbSNP_BitField_latest.pdf
"""
inVcfDir='/data/cellardata/users/btsui/dbsnp/Homo_sapiens/All_20170710.f1_byte2_not_00.vcf.gz' 
vcfDf=pd.read_csv(inVcfDir,sep='\t',header=None)
vcfDf.columns=['Chr','Pos','RsId','RefBase','AltBase','','','Annot']
vcfDf['Chr']=vcfDf['Chr'].astype(np.str)
my_chrom_vcfDf=vcfDf[(vcfDf['Chr']==queryChr)]
posToRefBase=my_chrom_vcfDf.set_index('Pos')['RefBase']
posToRefBase=posToRefBase.str[0]
posToRefBase=posToRefBase.groupby(posToRefBase.index).first()


/cellar/users/btsui/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
vcfDf.head()['Annot'].values

array(['RS=1045587;RSPOS=14727;RV;dbSNPBuildID=117;SSR=0;SAO=0;VP=0x050028040005000002000100;GENEINFO=DDX11L1:100287102|WASH7P:653635;WGT=1;VC=SNV;PM;PMC;R3;ASP',
       'RS=9783068;RSPOS=630825;dbSNPBuildID=119;SSR=1;SAO=0;VP=0x050028020005000002000140;GENEINFO=LOC101928626:101928626;WGT=1;VC=SNV;PM;PMC;R5;ASP',
       'RS=9701099;RSPOS=630833;dbSNPBuildID=119;SSR=1;SAO=0;VP=0x050028020005000002000140;GENEINFO=LOC101928626:101928626;WGT=1;VC=SNV;PM;PMC;R5;ASP',
       'RS=3094315;RSPOS=817186;RV;dbSNPBuildID=103;SSR=0;SAO=0;VP=0x05012802000515053f000101;GENEINFO=FAM87B:400728;WGT=1;VC=SNV;PM;PMC;SLO;R5;ASP;VLD;G5;HD;GNO;KGPhase1;KGPhase3;CAF=0.2817,0.7183;COMMON=1;TOPMED=0.351947,0.648053',
       'RS=12562034;RSPOS=833068;dbSNPBuildID=120;SSR=0;SAO=0;VP=0x050128080005170537000100;GENEINFO=LINC01128:643837;WGT=1;VC=SNV;PM;PMC;SLO;INT;ASP;VLD;G5A;G5;HD;GNO;KGPhase1;KGPhase3;CAF=0.8081,0.1919;COMMON=1'],
      dtype=object)

In [21]:
##slice out only the relevant positions
sra_dump_pickle_dir='/cellar/users/btsui/Data/SRA/DUMP/sra_dump.pickle'
technical_meta_data_df=pd.read_pickle(sra_dump_pickle_dir)

In [22]:
metaDataMappingSDir='/cellar/users/btsui/Data/nrnb01_nobackup/METAMAP//input/allAttrib.v5.csv.NCI.prefilter.pyc'
bio_metaDf=pd.read_pickle(metaDataMappingSDir)

### map the srr to srs

In [23]:
inMapDf=myChrPosDf.copy()
inMapDf.shape

(544557, 8)

In [24]:
srrToSrsS=technical_meta_data_df.set_index(['Run_db','Run_digits'])['Sample']

In [25]:
tmpDf3=inMapDf.set_index(['Run_db','Run_digits'],drop=False)

In [26]:
tmpDf3['Sample']=srrToSrsS.loc[tmpDf3.index].values

In [27]:
#vcfDf.head()
myIndexOrder2=['Chr','Pos','base']

In [28]:
mutlI=tmpDf3.set_index(myIndexOrder2).index

In [115]:
#technical_meta_data_df

In [30]:
vcfDf.shape

(393242, 8)

In [31]:
'ERR008901'

'ERR008901'

In [32]:
### need to map all the 

tmpDf3['RsId']=vcfDf.drop_duplicates(['Chr','Pos','AltBase']).set_index(['Chr','Pos','AltBase']).reindex(mutlI)['RsId'].values

In [33]:
tmpDf4=tmpDf3[tmpDf3['RsId'].notnull()].groupby(['RsId','Sample'])['ReadDepth'].sum().to_frame()

In [34]:
tmpDf4.index.get_level_values('RsId').nunique()

5

In [35]:
tmpDf4.unstack().T.head()

RsId                 rs104886015  rs121913225  rs121913226  rs121913373  \
          Sample                                                          
ReadDepth SRS000034          NaN          NaN        262.0          NaN   
          SRS000038          NaN          NaN         29.0          NaN   
          SRS000047          NaN          NaN        283.0          NaN   
          SRS000051          NaN          NaN        243.0          NaN   
          SRS000055          NaN          NaN         17.0          NaN   

RsId                 rs397516897  
          Sample                  
ReadDepth SRS000034        532.0  
          SRS000038         58.0  
          SRS000047        570.0  
          SRS000051        488.0  
          SRS000055         34.0

In [36]:
bio_metaDf[bio_metaDf.attrib=='molecular data type']

,srs,attrib,CUI,score,NCI,NciEng
27,SRS648841,molecular data type,C0205369,827,C38024,Specified
28,SRS648841,molecular data type,C0002085,827,C16277,Allele
214,SRS439430,molecular data type,C0444667,694,C25326,Whole
215,SRS439430,molecular data type,C3538927,861,C101296,Exome
231,SRS1360937,molecular data type,C0444667,694,C25326,Whole
232,SRS1360937,molecular data type,C0017428,861,C16629,Genome
240,SRS1786075,molecular data type,C0017431,861,C16631,Genotype
241,SRS1786075,molecular data type,C1527094,694,C41173,SNP Info
242,SRS1786075,molecular data type,C0752046,694,C18279,Single Nucleotide Polymorphism
310,SRS1889914,molecular data type,C0017431,827,C16631,Genotype


In [37]:
bio_metaDf.attrib.value_counts().head(n=50)

TITLE                               5678720
SCIENTIFIC NAME                     3292071
study name                          2841550
DESCRIPTION                         2737530
BioSampleModel                      2448286
sex                                 2078991
study design                        1968074
source name                         1922618
geo loc name                        1734993
SUBMITTER ID                        1683774
molecular data type                 1565807
isolation source                    1173499
tissue                              1039759
cell type                            949195
lat lon                              905507
gap accession                        828996
host                                 687152
strain                               676625
gap consent code                     664242
isolate                              663732
body site                            640742
biospecimen repository               610394
submitter handle                

In [38]:
bio_metaDf.attrib.value_counts()

TITLE                                 5678720
SCIENTIFIC NAME                       3292071
study name                            2841550
DESCRIPTION                           2737530
BioSampleModel                        2448286
sex                                   2078991
study design                          1968074
source name                           1922618
geo loc name                          1734993
SUBMITTER ID                          1683774
molecular data type                   1565807
isolation source                      1173499
tissue                                1039759
cell type                              949195
lat lon                                905507
gap accession                          828996
host                                   687152
strain                                 676625
gap consent code                       664242
isolate                                663732
body site                              640742
biospecimen repository            

In [39]:
accessions=pd.read_csv('/cellar/users/btsui/1000genomes.sequence.index',sep='\t',comment='#').iloc[:,3].unique()

/cellar/users/btsui/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
allSRSDir='/cellar/users/btsui/Data/nrnb01_nobackup/METAMAP/allSRS.pickle'

In [48]:
mySrsS=pd.read_pickle(allSRSDir)

In [99]:
#thousand_g_srp='SRP004054'
accessions=['SRP012682']# gtex
hit_techDf=technical_meta_data_df[technical_meta_data_df.Study.isin(accessions)]

In [100]:
hit_srs=hit_techDf['Sample'].unique()

In [101]:
mySrsS.head()

ERS069057  TITLE                                Streptococcus pneumoniae
           SCIENTIFIC_NAME                      Streptococcus pneumoniae
           SUBMITTER_ID          SN20648-sc-2011-11-03T19:59:41Z-1101437
           Strain                                                       
           Sample Description                                           
dtype: object

In [102]:
m_srs=mySrsS.index.get_level_values(0).isin(hit_srs)

In [103]:
%%time
subSrsS=mySrsS[m_srs]

CPU times: user 36 ms, sys: 20 ms, total: 56 ms
Wall time: 55.2 ms


In [104]:
attribVC=subSrsS.index.get_level_values(1).value_counts()

In [105]:
len(subSrsS)

231200

In [106]:

tmpSubS=subSrsS[subSrsS.index.get_level_values(1).isin(attribVC.index[attribVC>100])]

In [107]:
orderLevels=['level_0','level_1']
metaTable1000Genome=tmpSubS.reset_index().drop_duplicates(orderLevels).set_index(orderLevels)[0].unstack()#.groupby(level=[])

In [108]:
myL=[]
for col in metaTable1000Genome.columns:#.apply(lambda tmpS:tmpS.value_counts(),axis=0)
    myTmpVC=metaTable1000Genome[col].value_counts().iloc[:5]
    myL.append(myTmpVC)
    #print (myTmpVC)
    #print ("\n\n")

In [109]:
tmpS=pd.concat(myL,keys=metaTable1000Genome.columns)

In [114]:
tmpS

level_1                                                                                                                                                                                
SCIENTIFIC_NAME                   Homo sapiens                                                                                                                                             11036
TITLE                             Non-tumor DNA:DNA Genomic sample from Whole Blood of a human male participant in the dbGaP study "Genotype-Tissue Expression (GTEx)"                       784
                                  Non-tumor DNA:DNA Genomic sample from Whole Blood of a human female participant in the dbGaP study "Genotype-Tissue Expression (GTEx)"                     448
                                  Non-tumor RNA:Total RNA sample from Muscle - Skeletal of a human male participant in the dbGaP study "Genotype-Tissue Expression (GTEx)"                   303
                                  Non-tumor 

In [111]:
excel=pd.ExcelWriter('./tmpData/tmp.xlsx')
tmpS.to_frame().to_excel(excel)
excel.close()

In [112]:
!echo $PWD/./tmpData/tmp.xlsx

/cellar/users/btsui/Project/METAMAP/notebook/RapMapTest/Analysis/./tmpData/tmp.xlsx


In [113]:
#metaTable1000Genome['cell_type'].value_counts()

KeyError: 'cell_type'

In [ ]:
processed_I=myPosDf.set_index(['Run_digits']).index

In [ ]:
hit_techDf.set_index(['Run_digits']).index.isin(processed_I).sum()

In [97]:
#hit_techDf.Study.value_counts()